<a href="https://colab.research.google.com/github/iBotamon/yans_hakathon_2019/blob/master/yans_hackathon2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!cat /etc/issue
!echo "GPUか確認↓ (「ランタイム」→「ランタイムのタイプを変更」でGPUインスタンスに変更可能)"
!nvidia-smi
!echo "残り時間↓(0.5daysになるとインスタンスが再起動される)"
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
%cd /content # ホームディレクトリ


Ubuntu 18.04.2 LTS \n \l

GPUか確認↓ (「ランタイム」→「ランタイムのタイプを変更」でGPUインスタンスに変更可能)
Sun Aug 25 10:10:15 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    11W /  70W |     10MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:  

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# Google Driveにアクセス出来るようにする　
!ls /content/drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Google Driveにファイルを退避させる場合
!mkdir /content/drive/My\ Drive/yans_2019
# cpコマンドで退避できる
!cp sample_data/mnist_test.csv /content/drive/My\ Drive/yans_2019/
# Google Driveから移動する場合
!cp /content/drive/My\ Drive/yans_2019/train.csv sample_data/mnist_test.csv 

In [0]:
# gpu環境確認
# chainer
import chainer
chainer.print_runtime_info()
# pytorch
import torch
print('pytorch:')
print(torch.__version__)


In [0]:
!pip install gensim scikit-learn scipy

In [0]:
# gloveを使う例
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")
# https://github.com/RaRe-Technologies/gensim-data#models
model.most_similar("what")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('why', 0.9699443578720093),
 ('how', 0.9634339213371277),
 ('nothing', 0.9529852271080017),
 ('something', 0.9517911076545715),
 ('think', 0.9498581290245056),
 ('anything', 0.9433503150939941),
 ('know', 0.9372829794883728),
 ('we', 0.9367746114730835),
 ('fact', 0.9330813884735107),
 ('certainly', 0.9317318797111511)]

In [0]:
# ↓ファイル編集する場合 以下のように
%%writefile baseline.py
import csv
from gensim.utils import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.svm import LinearSVC

# load dataset

def load_dataset(filename):
    f = open(filename, 'r')
    reader = csv.reader(f)
    header = next(reader)
    lines = [row for row in reader]
    f.close()
    return lines

train = load_dataset(filename='train_100000.csv')
print('#train:', len(train))
# questions = [' '.join(list(tokenize(row[0], to_lower=True))) for row in lines] # tokenize
questions1 = [' '.join(list(tokenize(row[0], to_lower=True))) for row in train] # tokenize
questions2 = [' '.join(list(tokenize(row[1], to_lower=True))) for row in train] # tokenize
# questions1 = [row[0] for row in train] # no tokenize
# questions2 = [row[1] for row in train] # no tokenize
y = [int(row[-1]) for row in train]

# tf-idf classifior
vectorizer = TfidfVectorizer() 
questions = questions1 + questions2
vectorizer.fit(questions)
X1 = vectorizer.transform(questions1)
X2 = vectorizer.transform(questions2)

print('features:', vectorizer.get_feature_names())

X = hstack((X1, X2))
print('X:', X.shape)

# train classifier
print('train classifier:')
clf = LinearSVC().fit(X, y)

# predict
dev = load_dataset(filename='dev.csv')
questions1 = [' '.join(list(tokenize(row[0], to_lower=True))) for row in dev] # tokenize
questions2 = [' '.join(list(tokenize(row[1], to_lower=True))) for row in dev] # tokenize
y_dev_gold = [int(row[-1]) for row in dev]
X1 = vectorizer.transform(questions1)
X2 = vectorizer.transform(questions2)
X_dev = hstack((X1, X2))
print('X_dev:', X_dev.shape)
y_dev_predict = clf.predict(X_dev)
print('y_dev_predict:', y_dev_predict)
print('y_dev_gold:', y_dev_gold)

result = [predict == gold for predict, gold in zip(y_dev_predict, y_dev_gold)]

print('acuracy [dev]:', 100.0 * sum(result) / len(result))

Overwriting baseline.py


In [0]:
# ! head train.csv
% run baseline.py # 1分くらいで結果出るはず

#train: 100000
features: ['__', '___', '____', '_____', '______', '_______', '______when', '_ch_', '_main', '_mar', '_post', '_v', 'a_', 'a____in', 'a_i', 'a_n', 'aa', 'aaa', 'aaac', 'aaai', 'aac', 'aachen', 'aadam', 'aadhaar', 'aadhar', 'aadmi', 'aadvantage', 'aae', 'aafreen', 'aaft', 'aahat', 'aai', 'aaj', 'aakash', 'aalavandhan', 'aaloma', 'aalto', 'aam', 'aamdaar', 'aamir', 'aammir', 'aane', 'aang', 'aankho', 'aankhon', 'aansoo', 'aao', 'aap', 'aapke', 'aapl', 'aarathi', 'aarcitecture', 'aarid', 'aaron', 'aarp', 'aas', 'aashish', 'aata', 'aatmakatha', 'aau', 'aax', 'aayaa', 'aayan', 'aayiram', 'aayog', 'ab', 'ababa', 'abacus', 'abadan', 'abaday', 'abagnale', 'abaj', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abap', 'abaqus', 'abase', 'abasement', 'abatement', 'abaxis', 'abb', 'abbey', 'abbott', 'abbr', 'abbreviate', 'abbreviated', 'abbreviation', 'abbreviations', 'abc', 'abcd', 'abcdef', 'abco', 'abd', 'abdomen', 'abdomens', 'abdominal', 'abducted', 'abductee', 'abductee

In [0]:
# ↓ファイル編集する場合 以下のように 
%%writefile baseline_glove.py
import csv
from gensim.utils import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import numpy as np

# load glove vectors
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")
# https://github.com/RaRe-Technologies/gensim-data#models
# model.most_similar("what")


# load dataset

def load_dataset(filename):
    f = open(filename, 'r')
    reader = csv.reader(f)
    header = next(reader)
    lines = [row for row in reader]
    f.close()
    return lines

train = load_dataset(filename='train_100000.csv')
print('#train:', len(train))
questions1 = [list(tokenize(row[0], to_lower=True)) for row in train] # tokenize
questions2 = [list(tokenize(row[1], to_lower=True)) for row in train] # tokenize
# questions1 = [row[0] for row in train] # no tokenize
# questions2 = [row[1] for row in train] # no tokenize
y = [int(row[-1]) for row in train]

# glove vectors
# print(model["what"].shape)
def _to_vec(words, model):
    if len([w for w in words if w in model]):
        return np.sum([model[w] for w in words if w in model], axis=0).reshape(1, 50)
    else:
        return np.zeros((1, 50))

X1 = np.concatenate([_to_vec(words, model) for words in questions1], axis=0)
X2 = np.concatenate([_to_vec(words, model) for words in questions2], axis=0)
print('X1:', X1.shape)
print('X2:', X2.shape)
# vectorizer = TfidfVectorizer() 
# questions = questions1 + questions2
# vectorizer.fit(questions)
# X1 = vectorizer.transform(questions1)
# X2 = vectorizer.transform(questions2)

# print('features:', vectorizer.get_feature_names())

X = np.concatenate([X1, X2], axis=1)
print('X:', X.shape)

# train classifier
print('train classifier:')
clf = MLPClassifier().fit(X, y)

# predict
dev = load_dataset(filename='dev.csv')

questions1 = [list(tokenize(row[0], to_lower=True)) for row in dev] # tokenize
questions2 = [list(tokenize(row[1], to_lower=True)) for row in dev] # tokenize
y_dev_gold = [int(row[-1]) for row in dev]
X1 = np.concatenate([_to_vec(words, model) for words in questions1], axis=0)
X2 = np.concatenate([_to_vec(words, model) for words in questions2], axis=0)
X_dev = np.concatenate([X1, X2], axis=1)
print('X_dev:', X_dev.shape)
y_dev_predict = clf.predict(X_dev)
print('y_dev_predict:', y_dev_predict)
print('y_dev_gold:', y_dev_gold)

result = [predict == gold for predict, gold in zip(y_dev_predict, y_dev_gold)]

print('acuracy [dev]:', 100.0 * sum(result) / len(result))

# % run baseline_glove.py # 5分くらい

Overwriting baseline_glove.py


In [0]:
% run baseline_glove.py # 20分くらい

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#train: 100000
X1: (100000, 50)
X2: (100000, 50)
X: (100000, 100)
train classifier:


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


X_dev: (5000, 100)
y_dev_predict: [0 0 0 ... 0 1 0]
y_dev_gold: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,